In [1]:
!pip install git+https://github.com/The-AI-Alliance/gofannon.git --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 29.4 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
from google.colab import userdata
import json
# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=userdata.get('deepinfra'),
    base_url="https://api.deepinfra.com/v1/openai",
)

from gofannon.basic_math.exponents import Exponents
from gofannon.basic_math.multiplication import Multiplication
from gofannon.basic_math.division import Division
from gofannon.basic_math.addition import Addition
from gofannon.basic_math.subtraction import Subtraction


tool_list =  [F() for F in [Addition, Subtraction, Multiplication, Division, Exponents]]
tool_map = {f.name: f.fn for f in tool_list}
tool_desc_list = [f.definition for f in tool_list]

In [3]:
# here is the user request
messages = [
    {
        "role": "user",
        "content": "What is 12*457?"
    }
]

# let's send the request and print the response
response = openai.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
    messages=messages,
    tools=tool_desc_list,
    tool_choice="auto",
)
tool_calls = response.choices[0].message.tool_calls
for tool_call in tool_calls:
    print(tool_call.model_dump())


{'id': 'call_YDinihVWmM4fgG0G7ZrD8JoI', 'function': {'arguments': '{"num1": 12.0, "num2": 457.0}', 'name': 'multiplication'}, 'type': 'function'}


In [4]:
# extend conversation with assistant's reply
messages.append(response.choices[0].message)

for tool_call in tool_calls:
  function_name = tool_call.function.name
  function_args = json.loads(tool_call.function.arguments)
  function_response = tool_map[function_name](**function_args)

  # extend conversation with function response
  messages.append({
      "tool_call_id": tool_call.id,
      "role": "tool",
      "content": function_response,
  })


# get a new response from the model where it can see the function responses
second_response = openai.chat.completions.create(
  model="meta-llama/Meta-Llama-3.1-70B-Instruct",
  messages=messages,
  tools=tool_desc_list,
  tool_choice="auto",
)

print(second_response.choices[0].message.content)


The result of the multiplication is 5484.
